<a id='sect0'></a>
## <font color='darkblue'>Preface</font>
雖然我年紀已經不小, 但是追朔 [FP (Functional programming) 的歷史](https://en.wikipedia.org/wiki/Functional_programming#History), 我也只能算年輕:
> The lambda calculus, developed in the 1930s by Alonzo Church, is a formal system of computation built from function application. 

<br/><br/>


既然歷史攸久, 因此一言難盡, 在這邊只會吹吹皮毛, 希望至少可以 cover 下面的內容:
* <font size='3ptx'><b><a href='#sect1'>Basic FP terminology (Function/Side effect/Closure ...)</a></b></font>
* <font size='3ptx'><b><a href='#sect2'>Lambda 用法 & 範例</a></b></font>
* <font size='3ptx'><b><a href='#sect3'>常常和 Lambda 一起使用的其他函數 map/filter/functools.reduce</a></b></font>
* 可以接受的 Lambda 使用時機
* Review 時會被打槍的用法
* FPU 簡介

In [5]:
#!pip install fpu

In [19]:
from fpu.flist import *
from functools import partial
from typing import Sequence

<a id='sect1'></a>
## <font color='darkblue'>Basic FP terminology</font>
* <font size='3ptx'><b><a href='#sect1_1'>FP Terminology - Imperative vs Declarative</a></b></font>
* <font size='3ptx'><b><a href='#sect1_2'>FP Terminology - Closure</a></b></font>
* <font size='3ptx'><b><a href='#sect1_3'>FP Terminology - Currying</a></b></font>
<br/>

Functional programming has a [long history](https://en.wikipedia.org/wiki/Functional_programming#History). In a nutshell, **its a style of programming where you focus on transforming data through the use of small expressions that ideally don’t contain side effects.** In other words, when you call <font color='blue'>my_fun(1, 2)</font>, it will always return the same result. This is achieved by **immutable data** typical of a functional language.

![function & side effect](images/1.PNG)

([image source](https://www.fpcomplete.com/blog/2017/04/pure-functional-programming/))
<br/>

<a id='sect1_1'></a>
### <font color='darkgreen'>FP Terminology - Imperative vs Declarative</font>
你可以把 `Imperative` 與 `Decleartive` 想成將編程語言分類的一種方法. (<font color='brown'>顏色, 大小, 形狀 etc</font>). 稍後會說明這兩種陣營的語言寫起來的差別:

![function & side effect](images/2.PNG)
<br/>

#### <font size='3ptx'>Imperative</font>
底下是 [Imperative 語言的 wiki 說明](https://en.wikipedia.org/wiki/Imperative_programming):
> **Imperative programming** is like building assembly lines, which take some initial global state as raw material, apply various specific transformations, mutations to it as this material is pushed through the line, and at the end comes off the end product, the final global state, that represents the result of the computation. Each step needs to change, rotate, massage the workpiece precisely one specific way, so that it is prepared for subsequent steps downstream. Every step downstream depend on every previous step, and their order is therefore fixed and rigid. Because of these dependencies, an individual computational step has not much use and meaning in itself, but only in the context of all the others, and to understand it, one must understand how the whole line works.
<br/>

現今大部分的語言都屬於 imperative 陣營, 看描述很辛苦, 透過比對兩者的編程 style 可以很容易發現兩者不同. 底下是 imperative style 的編成方式:

In [7]:
salaries = [
    (True, 9000),   # (Is female, salary)
    (False, 12000), 
    (False, 6000),
    (True, 14000),
]

In [23]:
def imperative_way(salaries):
    '''Gets the sum of salaries of female and male.
    
    Args:
        salaries: List of salary. Each element is of tuple(is_female: bool, salary: int)
    
    Returns:
        Tuple(Sum of female salaries, Sum of male salaries)
    '''
    female_sum = male_sum = 0
    for is_female, salary in salaries:
        if is_female:
            female_sum += salary
        else:
            male_sum += salary
            
    return (female_sum, male_sum)

In [3]:
imperative_way(salaries)

(23000, 18000)

#### <font size='3ptx'>Declarative</font>
底下是 [Declarative 語言的 wiki 說明](https://en.wikipedia.org/wiki/Declarative_programming):
> A style of building the structure and elements of computer programs—that expresses the logic of a computation without describing its control flow.

In [24]:
def add(a, b):
    return a + b

def salary_sum(is_female: bool):
    '''Return calculator to sum up the salary based on female/male
    
    Args:
        is_female: True to return calculator to sum up salaries of female
            False to return calculator to sum up salaries of male.
            
    Returns:
        Calculator to sum up salaries.
    '''
    def _salary_sum(salaries):
        flist = fl(salaries)
        return flist.filter(lambda t: t[0] == is_female) \
                    .map(lambda t: t[1]) \
                    .foldLeft(0, add)
    return _salary_sum

In [10]:
def declarative_way(salaries):
    return (
        salary_sum(is_female=True)(salaries),  # Salary sum of female
        salary_sum(is_female=False)(salaries), # Salary sum of male
    )

In [11]:
declarative_way(salaries)

(23000, 18000)

<a id='sect1_2'></a>
### <font color='darkgreen'>FP Terminology - Closure</font> ([back](#sect1))
A [**Closure**](https://en.wikipedia.org/wiki/Closure_(computer_programming)) is a function which **simply creates a scope that allows the function to access and manipulate the variables in enclosing scopes**. Normally, you will follow below steps to create a Closure in Python:
* We have to create a nested function (a function inside another function).
* This nested function has to refer to a variable defined inside the enclosing function.
* The enclosing function has to return the nested function

<br/>

簡單說就是你的 Function object 綁定一個封閉的 name space (<font color='brown'>[這篇](https://dboyliao.medium.com/%E8%81%8A%E8%81%8A-python-closure-ebd63ff0146f)介紹還蠻清楚, 可以參考</font>), 直接來看範例理解:

In [15]:
def contain_N(n):
    def _inner(sequence: Sequence):
        return n in sequence
    
    return _inner

In [16]:
contain_5 = contain_N(5)
contain_10 = contain_N(10)

In [18]:
my_datas = [1, 2, 3, 4, 5]
print(f'my_data={my_datas} contains 5? {contain_5(my_datas)}')
print(f'my_data={my_datas} contains 10? {contain_10(my_datas)}')

my_data=[1, 2, 3, 4, 5] contains 5? True
my_data=[1, 2, 3, 4, 5] contains 10? False


上面的函數 `contain_N` 返回一個 closure. 該 closure 綁訂了變數 `n`. (<font color='blue'>contain_5</font> <font color='brown'>綁定的 `n` 為 5;</font> <font color='blue'>contain_10</font> <font color='brown'>綁定的 `n` 為 10</font>)

<a id='sect1_3'></a>
### <font color='darkgreen'>FP Terminology - Currying</font> ([back](#sect1))
底下是 wiki 上對 [**currying**](https://en.wikipedia.org/wiki/Curry_(programming_language)) 的說明:
> <b>Currying is like a kind of incremental binding of function arguments</b>. It is the technique of breaking down the evaluation of a function that takes multiple arguments into evaluating a sequence of single-argument functions.

<br/>

很可惜在 Python 預設的函數並不支援這個特性, 幸運的是在模組 [**functools**](https://docs.python.org/3/library/functools.html) 有提供 [partial](https://docs.python.org/3/library/functools.html#functools.partial) 來模擬 currying 的好處. 直接來看範例: 

In [41]:
def sum_salary_by_sex(*args):
    def _sum_salary_by_sex(is_female: bool, salaries: Sequence) -> int:
        return sum(map(lambda t: t[1], filter(lambda t: t[0]==is_female, salaries)))
    
    if len(args) == 1:
        return partial(_sum_salary_by_sex, args[0])
    
    return _sum_salary_by_sex(*args)

In [42]:
# Get female salaries
sum_salary_by_sex(True, salaries)

23000

In [43]:
# # Get female salaries in currying way
sum_salary_by_sex(True)(salaries)

23000

In [39]:
# Get male salaries
sum_salary_by_sex(False, salaries)

18000

In [45]:
# Get male salaries in curring way
sum_salary_by_sex(False)(salaries)

18000

In [46]:
sum_salary_by_female = sum_salary_by_sex(True)
sum_salary_by_male = sum_salary_by_sex(False)

In [47]:
sum_salary_by_female(salaries)

23000

In [48]:
sum_salary_by_male(salaries)

18000

我們透過 currying 的特性便可以得到新的函數 <font color='blue'>sum_salary_by_female</font> 與 <font color='blue'>sum_salary_by_male</font>. 是不是很方便?

<a id='sect2'></a>
## <font color='darkblue'>Lambda 用法 & 範例</font> ([back](#sect0))
[**Lambda**](https://docs.python.org/3/reference/expressions.html#lambda) 在 Python 是一個關鍵字用來定義 匿名函數. 底下是使用 lambda 匿名函數的注意事項:
* It can only contain expressions and can’t include statements ([No Statesments](#sect2_1)) in its body.
* [It is written as a single line of execution](#sect2_2).
* [It does not support type annotations.](#sect2_3)
* [It can be immediately invoked](#sect2_4) ([IIFE](https://en.wikipedia.org/wiki/Immediately_invoked_function_expression)).

<a id='sect2_1'></a>
### <font color='darkgreen'>No Statements</font>
<b><font size='3ptx' color='darkred'>A lambda function can’t contain any statements</font></b>. In a lambda function, statements like `return`, `pass`, `assert`, or `raise` will raise a [**SyntaxError**](https://realpython.com/invalid-syntax-python/) exception. Here’s an example of adding assert to the body of a lambda:
```python
>>> (lambda x: assert x == 2)(2)
  File "<input>", line 1
    (lambda x: assert x == 2)(2)
                    ^
SyntaxError: invalid syntax
```
<br/>

<a id='sect2_2'></a>
### <font color='darkgreen'>Single Expression</font>
<font size='3ptx'><b>In contrast to a normal function, a Python lambda function is a single expression</b></font>. Although, in the body of a lambda, you can spread the expression over several lines using parentheses or a multiline string, it remains a single expression:
```python
>>> (lambda x:
... (x % 2 and 'odd' or 'even'))(3)
'odd'
```
<br/>

The example above returns the string 'odd' when the lambda argument is odd, and 'even' when the argument is even. It spreads across two lines because it is contained in a set of parentheses, but it remains a single expression.

<a id='sect2_3'></a>
### <font color='darkgreen'>Type Annotations</font>
If you’ve started adopting type hinting, which is now available in Python, then you have another good reason to prefer normal functions over Python lambda functions. Check out [**Python Type Checking** (Guide)]((https://realpython.com/python-type-checking/#hello-types)) to get learn more about Python type hints and type checking. In a lambda function, there is no equivalent for the following:
```python
def full_name(first: str, last: str) -> str:
    return f'{first.title()} {last.title()}'
```
<br/>

Any type error with <font color='blue'>full_name()</font> can be caught by tools like [**mypy**](http://mypy-lang.org/) or [**pyre**](https://pyre-check.org/), whereas a [**SyntaxError**](https://realpython.com/invalid-syntax-python/) with the equivalent lambda function is raised at runtime:
```python
>>> lambda first: str, last: str: first.title() + " " + last.title() -> str
  File "<stdin>", line 1
    lambda first: str, last: str: first.title() + " " + last.title() -> str

SyntaxError: invalid syntax
```
<br/>

Like trying to include a statement in a lambda, adding type annotation immediately results in a [**SyntaxError**](https://realpython.com/invalid-syntax-python/) at runtime.

<a id='sect2_4'></a>
### <font color='darkgreen'>IIFE</font>
You’ve already seen several examples of [immediately invoked function execution](https://developer.mozilla.org/en-US/docs/Glossary/IIFE):
```python
>>> (lambda x: x * x)(3)
9
```
<br/>

<b>It’s a direct consequence of a lambda function being callable as it is defined</b>. For example, this allows you to pass the definition of a Python lambda expression to a higher-order function like [map()](https://docs.python.org/3/library/functions.html#map), [filter()](https://docs.python.org/3/library/functions.html#filter), or [**functools**.reduce()](https://docs.python.org/3/library/functools.html#functools.reduce), or to a `key function`.

<a id='sect3'></a>
## <font color='darkblue'>常常和 Lambda 一起使用的其他函數 map/filter/functools.reduce</font> ([back](#sect0))

## <font color='darkblue'>Supplement</font>
* [Medium - 聊聊 Python Closure](https://dboyliao.medium.com/%E8%81%8A%E8%81%8A-python-closure-ebd63ff0146f)